# Trend Screener

In [36]:
pip install "yfinance[nospam]" stocktrends matplotlib pandas requests_cache

Note: you may need to restart the kernel to use updated packages.


In [37]:
import yfinance as yf
import pandas as pd
import stocktrends as st
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
import traceback
import requests_cache

session = requests_cache.CachedSession('yfinance.cache')
session.headers['User-agent'] = 'Mozilla/5.0 (compatible; MSIE 7.0; Windows; U; Windows NT 10.4; Win64; x64; en-US Trident/4.0)'

results = []

tickers_txt_path = 'us_stock2.txt'

try:
    with open(tickers_txt_path, 'r') as file:
        tickers = [line.strip() for line in file if line.strip()]
except Exception as e:
    print(f"Error reading tickers from text file: {e}")
    tickers = []

start = '2023-03-26'
end = '2024-07-26'

data = yf.download(tickers, start = start, end = end, session=session)

print(data)

[*********************100%%**********************]  99 of 99 completed

16 Failed downloads:
['ACAXR', 'AAIC', 'ABCM', 'ACAXU', 'AAIN', 'AAU', 'ACRX', 'ACAH', 'ACAX', 'ACRO', 'ACOR', 'ACDCW']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')
['ABLLW', 'ACER', 'ACONW']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")
['ACBA']: YFChartError('%ticker%: No data found, symbol may be delisted')


Price        Adj Close                                                         \
Ticker               A         AA  AACG   AACI  AACIU    AACT  AADI AAIC AAIN   
Date                                                                            
2023-03-27  133.070007  40.119999  1.82  10.07  10.00     NaN  7.25  NaN  NaN   
2023-03-28  132.630005  40.529999  1.81  10.07  10.00     NaN  6.85  NaN  NaN   
2023-03-29  133.850006  40.939999  1.83  10.07  10.00     NaN  7.12  NaN  NaN   
2023-03-30  134.490005  41.759998  1.87  10.08  10.00     NaN  7.11  NaN  NaN   
2023-03-31  138.339996  42.560001  1.93  10.08  10.00     NaN  7.24  NaN  NaN   
...                ...        ...   ...    ...    ...     ...   ...  ...  ...   
2024-07-19  131.779999  34.919998  0.65  11.39  11.93  10.700  1.43  NaN  NaN   
2024-07-22  133.419998  34.529999  0.65  11.35  11.93  10.700  1.51  NaN  NaN   
2024-07-23  130.649994  33.900002  0.67  11.30  11.93  10.715  1.57  NaN  NaN   
2024-07-24  134.160004  32.8

In [42]:
def calculate_trend(df, ticker):
    df[f'{ticker} 50MA'] = df['Adj Close'].rolling(window=50).mean()
    df[f'{ticker} 150MA'] = df['Adj Close'].rolling(window=150).mean()
    df[f'{ticker} 200MA'] = df['Adj Close'].rolling(window=200).mean()
    
    df[f'{ticker} 52W_High'] = df['Adj Close'].rolling(window=252).max()
    
    df[f'{ticker} 52W_High_Ratio'] = df['Adj Close'] / df[f'{ticker} 52W_High']
    
    df[f'{ticker} MA_Uptrend'] = (
        (df['Adj Close'] > df[f'{ticker} 50MA']) &
        (df[f'{ticker} 50MA'] > df[f'{ticker} 150MA']) &
        (df[f'{ticker} 150MA'] > df[f'{ticker} 200MA']) &
        (df[f'{ticker} 50MA'].diff() > 0) &
        (df[f'{ticker} 150MA'].diff() > 0) &
        (df[f'{ticker} 200MA'].diff() > 0)
    )

    df[f'{ticker} 200MA_Uptrend'] = df[f'{ticker} 200MA'].diff() > 0
    df[f'{ticker} 200MA_1Month_Uptrend'] = df[f'{ticker} 200MA_Uptrend'].rolling(window=20).mean() * 100
    df[f'{ticker} 200MA_4Month_Uptrend'] = df[f'{ticker} 200MA_Uptrend'].rolling(window=80).mean() * 100

    return df

results = {}
for ticker in tickers:
    try:
        ticker_data = data['Adj Close'][ticker]
        ticker_df = pd.DataFrame(ticker_data)
        ticker_df = ticker_df.rename(columns={ticker: 'Adj Close'})
        ticker_df = calculate_trend(ticker_df, ticker)

        results[ticker] = ticker_df
    except Exception as e:
        print(f"Error calculating moving averages for {ticker}: {e}")

for ticker, res_df in results.items():
    print(f"\n{ticker} Moving Averages and Uptrend Status:\n", res_df.tail())


EBS Moving Averages and Uptrend Status:
             Adj Close  EBS 50MA  EBS 150MA  EBS 200MA  EBS 52W_High  \
Date                                                                  
2024-07-19      11.20    6.8400   3.813733    3.44145         12.11   
2024-07-22      11.46    6.9810   3.874867    3.48380         12.11   
2024-07-23      12.49    7.1434   3.941267    3.53160         12.49   
2024-07-24      13.81    7.3326   4.017467    3.58580         13.81   
2024-07-25      13.80    7.5014   4.093867    3.63985         13.81   

            EBS 52W_High_Ratio  EBS MA_Uptrend  EBS 200MA_Uptrend  \
Date                                                                
2024-07-19            0.924856            True               True   
2024-07-22            0.946325            True               True   
2024-07-23            1.000000            True               True   
2024-07-24            1.000000            True               True   
2024-07-25            0.999276            True